# Fix walking and biking skims

There are two Census tracts that are disconnected from the biking/walking network (and thus most likely from the transit network as well): 2372 (06037980031), which is Terminal Island, and 3737 (06071010417), which is Palm Wells, CA. It is actually not possible to bike, walk, or take transit to Terminal Island - there are no bus stops, and the island is connected to the mainland only by three freeway bridges. It is theoretically possible to walk or bike to Palm Wells, but it's really in the middle of nowhere so assume no one does. There is one transit route to Palm Wells, operated by the [MBTA](https://www.transit.land/routes/r-9qj1-12#summary) (not the one you think), but we don't have that agency in our transit network anyhow.

In [ ]:
import numpy as np
import openmatrix as omx

In [ ]:
skims = omx.open_file('../model_inputs/skims.omx', mode='a')

In [ ]:
walk = np.array(skims['DISTWALK'])

In [ ]:
walk[:,3737]

In [ ]:
# set these to very high travel times, to effectively make their probability 0 (since TT has a negative coef)
walk[2372,:] = 42_000
walk[:,2372] = 42_000
walk[3737,:] = 42_000
walk[:,3737] = 42_000

In [ ]:
# but allow within-tract trips
avg_within_tract_dist = np.mean(np.diag(walk)[np.r_[:2372,2373:3737,3738:walk.shape[0]]])
walk[2372,2372] = avg_within_tract_dist
walk[3737,3737] = avg_within_tract_dist

In [ ]:
assert np.min(walk) >= 0

In [ ]:
skims.remove_node('/data/DISTWALK')

In [ ]:
skims['DISTWALK'] = walk

In [ ]:
# and for biking
bike = np.array(skims['DISTBIKE'])

In [ ]:
bike[2372,:] = 42_000
bike[:,2372] = 42_000
bike[3737,:] = 42_000
bike[:,3737] = 42_000

In [ ]:
avg_within_tract_dist = np.mean(np.diag(bike)[np.r_[:2372,2373:3737,3738:bike.shape[0]]])
bike[2372,2372] = avg_within_tract_dist
bike[3737,3737] = avg_within_tract_dist

In [ ]:
assert np.min(bike) >= 0

In [ ]:
skims.remove_node('/data/DISTBIKE')

In [ ]:
skims['DISTBIKE'] = bike

In [ ]:
skims.close()